In [1]:
import allel
import pandas as pd 
import numpy as np
from numba import jit
from typing import Tuple

In [2]:
rng = np.random.default_rng(21)

In [3]:
cd ~/ucsc-cgl/synth-genes

/home/apoirel/ucsc-cgl/synth-genes


In [4]:
variants_vcf = 'data/raw/BreastCancer.shuffle.vcf'

In [5]:
K = 6

## Ordinal encoding of haplotypes

In [6]:
@jit(nopython=True)
def _encode_haplotypes(
 variants_pos: np.ndarray, haplos: np.ndarray, n_samples: int, n_loci: int,
) -> np.ndarray:
    haplos_encoded = np.full((n_samples, n_loci), 0)
    haplos_idx = 0 
    for k, pos in enumerate(np.unique(variants_pos)):
        n_variants = np.sum(variants_pos == pos)
        for idx in range(1, n_variants+1):
            res = np.where(
                haplos[:, haplos_idx] == 1, idx, haplos_encoded[:,k]
            )
            haplos_encoded[:,k] = res
            haplos_idx += 1
    return haplos_encoded

### Test data

In [7]:
test_variants = pd.DataFrame({
    'POS': np.array([1, 2, 2, 3, 3, 3]),
    'REF': np.array(['A', 'GA', 'GA', 'T', 'T', 'T']),
    'ALT_1': np.array(['G', 'G', 'C', 'TA', 'C', 'G'])
})

In [8]:
test_haplos = np.array([
    [1, 0, 0, 0], [0, 1, 0, 1], [0, 0, 1, 0], 
    [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]
]).T

In [9]:
test_haplos

array([[1, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 1],
       [0, 1, 0, 0, 1, 0]])

In [10]:
expected_encoding = np.array([
    [1, 0, 0],
    [0, 1, 1],
    [0, 2, 3],
    [0, 1, 2],
])

In [11]:
_encode_haplotypes(test_variants['POS'].values, test_haplos, 4, 3)

array([[1, 0, 0],
       [0, 1, 1],
       [0, 2, 3],
       [0, 1, 2]])

### Real data

pre-processing

In [12]:
variants = (allel
	.vcf_to_dataframe(variants_vcf, fields=['POS', 'REF', 'ALT'])
	.drop(['ALT_2', 'ALT_3'], axis=1) # ALT_2, ALT_3 are always empty
)
genotypes = allel.read_vcf(variants_vcf, fields=['calldata/GT'])
genotypes = genotypes['calldata/GT']	
# scikit-allel reads missing values as -1
genotypes = np.where(genotypes == -1, 0, genotypes)
haplo_1 = genotypes[:,:,0]
haplo_2 = genotypes[:,:,1]
haplos = np.hstack((haplo_1, haplo_2)).T

In [13]:
n_variants_pos = (variants
    .groupby('POS')
    .count()['REF']
    .values) + 1

In [14]:
max_n_variants = np.sort(n_variants_pos)[-1]
n_loci = len(variants['POS'].unique())
n_samples = haplos.shape[0] 

In [15]:
haplos = _encode_haplotypes(variants['POS'].values, haplos, n_samples, n_loci)

## EM implementation

In [19]:
@jit(nopython=True, fastmath=True)
def _em_loop(
	n_iterations: int,
	K: int,
	n_samples: int,
	n_loci: int,
	n_variants_pos: np.ndarray,
	group_e: np.ndarray,
	group_probs: np.ndarray, 
	variant_probs: np.ndarray, 
	haplotypes: np.ndarray,
	logsum_approx: bool,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:	

	flattened_offset = np.arange(0, n_loci*2, 2)
 
	for i in range(n_iterations):
		print('Starting iteration ', i, ' /', n_iterations, '\n')
		for r in range(n_samples):
			# NOTE might cause problems for variant probs that are 0?
			log_probs_alpha = np.array([
				np.sum(log_variant_probs[alpha].flatten()[flattened_offset + haplotypes[r]]) 
				# workaround for not being able to use more than one advanced index in numba
				# normally would use variant_probs[alpha, np.arange(n_loci), haplos[r]]
				for alpha in range(K)
			]) 
			denominator = 0
			if logsum_approx: # use approximation for log of a sum
				denominator = logsum_approx(group_probs * log_probs_alpha)
			else:
				denominator = np.log(np.sum(group_probs * np.exp(log_probs_alpha)))     
			new_group_e = np.log(group_probs) + log_probs_alpha - denominator
			new_group_e = np.exp(new_group_e)
			if logsum_approx: # rescale to sum up to 1 if using an approximationk
				new_group_e = new_group_e / np.sum(new_group_e)
			if np.isinf(np.max(new_group_e)):
				raise ValueError('Infinite expectation')
			else:
				group_e[r,:] = new_group_e

		group_probs = np.sum(group_e, axis=0) / n_samples
		print('Group probabilities: ', group_probs)
		# variant probabilities update
		for alpha in range(K):
			norm_ct = group_probs[alpha] * n_samples
			for k in range(n_loci):
				for i in range(n_variants_pos[k]): # ignore placeholders in array
					variant_samples = np.where(haplotypes[:,k] == i, True, False) # find samples with given variant
					new_prob = np.sum(group_e[variant_samples, alpha]) / norm_ct
					variant_probs[alpha, k, i] = max(new_prob, 10e-12) # prevent issues from taking log(0)
		print('Variant probabilities: ', variant_probs[:2, :10])    
	return (group_probs, group_e, variant_probs)

In [20]:
# em initialization
group_e_ini = rng.random(size=(n_samples, K))
group_e = group_e_ini / np.sum(group_e_ini, axis=1, keepdims=1)
group_ini = rng.random(size=6)
group_probs = group_ini / np.sum(group_ini) # make this a probability vector
variant_ini = rng.random(size=(K, n_loci, max_n_variants)) 
variant_probs = variant_ini / np.sum(variant_ini, axis=2, keepdims=1)

In [21]:
res = _em_loop(
    10,
    K,
    n_samples,
    n_loci,
    n_variants_pos,
    group_e,
    group_probs,
    variant_probs,
    haplos,
    logsum_approx=True,
)

Starting iteration  0  / 10 

Group probabilities:  [1.95087429e-09 4.00500970e-45 9.99999998e-01 2.96406469e-19
 5.71840112e-46 2.57414745e-50]
Variant probabilities:  [[[1.00000000e+00 1.00000000e-11]
  [1.00000000e+00 1.00000000e-11]
  [1.00000000e+00 1.00000000e-11]
  [1.00000000e+00 1.00000000e-11]
  [9.99999988e-01 1.17308450e-08]
  [9.98119927e-01 1.88007345e-03]
  [9.99999989e-01 1.12758570e-08]
  [9.99999999e-01 7.13880736e-10]
  [9.99999999e-01 1.34234618e-09]
  [9.99999999e-01 7.09584936e-10]]

 [[9.99999999e-01 1.44524310e-09]
  [1.00000000e+00 1.26837208e-10]
  [9.99999993e-01 7.21219513e-09]
  [1.00000000e+00 1.00000000e-11]
  [1.00000000e+00 1.00000000e-11]
  [9.99999986e-01 1.35791637e-08]
  [9.99999437e-01 5.63082475e-07]
  [9.99998321e-01 1.67903069e-06]
  [1.00000000e+00 1.00000000e-11]
  [1.00000000e+00 1.00000000e-11]]]
Starting iteration  1  / 10 

Group probabilities:  [5.28067573e-05 9.56899874e-33 9.99947193e-01 3.12895154e-11
 1.97648669e-36 1.56769252e-44]
Va